In [3]:

%load_ext autoreload
%autoreload 2
from dataloader.dataloader import DataLoader
from model_selection.model_selection import ModelSelection
from performance_analysis.performance_analysis import PerformanceAnalysis
from feature_selection.feature_selection import FeatureSelector
from feature_extraction.feature_extraction import FeatureExtractor
from imports import *

In [1]:
classes = [
"Wool",
"Denim",
"Leather",
"Cotton",
"Silk"
    ]

In [4]:
# dump classes to pkl file
with open("texture_classes.pkl", "wb") as f:
    pickle.dump(classes, f)

## Data Loading

In [ ]:
dataloader = DataLoader(Path('./data'))

x_train, y_train, x_test, y_test, x_val, y_val = dataloader.load_data()

In [ ]:
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
# show random image from train and corresponding class
index = random.randint(0, len(x_train) - 1)
print(index)

print(x_train[index])
# plt.imshow(x_train[index])
# plt.show()
print(f"class= {classes[y_train[index]]}")

## Features Extraction And Selection

### GLCM Features

In [ ]:
feature_extractor = FeatureExtractor()

glcm_features_train = feature_extractor.extract_glcm_features(x_train)
glcm_features_val = feature_extractor.extract_glcm_features(x_val)

print(f"extracted_features shape: {glcm_features_train.shape}")
print(f"extracted_features_val shape: {glcm_features_val.shape}")

### Daisy Features

In [ ]:
daisy_features_train = feature_extractor.extract_daisy_features(x_train)
daisy_features_val = feature_extractor.extract_daisy_features(x_val)

print(f"extracted_features shape: {daisy_features_train.shape}")
print(f"extracted_features_val shape: {daisy_features_val.shape}")

### PCA

In [ ]:
feature_selector = FeatureSelector()

pca_features_train = feature_selector.extract_pca_features(daisy_features_train, load=False, num_pca_components=10)
pca_features_val = feature_selector.extract_pca_features(daisy_features_val, load=True, num_pca_components=10)

pca_features_train = np.concatenate((pca_features_train, glcm_features_train), axis=1)
pca_features_val = np.concatenate((pca_features_val, glcm_features_val), axis=1)

print(f"extracted_features shape: {pca_features_train.shape}")
print(f"extracted_features_val shape: {pca_features_val.shape}")

### Normalize PCA Features

In [111]:
extracted_features_train_mean = pca_features_train.mean(axis=0)
extracted_features_train_std = pca_features_train.std(axis=0)

# save these 2 to files
np.save("extracted_features_train_mean.npy", extracted_features_train_mean)
np.save("extracted_features_train_std.npy", extracted_features_train_std)

In [112]:
pca_features_train = (pca_features_train - extracted_features_train_mean) /extracted_features_train_std
pca_features_val = (pca_features_val - extracted_features_train_mean) /extracted_features_train_std

## Models

In [ ]:
model_selection = ModelSelection(pca_features_train, y_train, pca_features_val, y_val)

svm, pred_train, pred_val = model_selection.SVM()
performance_analysis = PerformanceAnalysis('SVM Train', pred_train, y_train)
performance_analysis.calculate_performance_metrics()
performance_analysis = PerformanceAnalysis('SVM VALIDATION', pred_val, y_val, True)
performance_analysis.calculate_performance_metrics()


knn, pred_train, pred_val = model_selection.KNN()
performance_analysis = PerformanceAnalysis('KNN Train', pred_train, y_train)
performance_analysis.calculate_performance_metrics()
performance_analysis = PerformanceAnalysis('KNN VALIDATION', pred_val, y_val, True)
performance_analysis.calculate_performance_metrics()

ensemble, pred_train, pred_val = model_selection.Ensemble()
performance_analysis = PerformanceAnalysis('Ensemble Train', pred_train, y_train)
performance_analysis.calculate_performance_metrics()
performance_analysis = PerformanceAnalysis('Ensemble VALIDATION', pred_val, y_val, True)
performance_analysis.calculate_performance_metrics()


adaboost, pred_train, pred_val = model_selection.AdaBoost()
performance_analysis = PerformanceAnalysis('AdaBoost Train', pred_train, y_train)
performance_analysis.calculate_performance_metrics()
performance_analysis = PerformanceAnalysis('AdaBoost VALIDATION', pred_val, y_val, True)
performance_analysis.calculate_performance_metrics()

In [ ]:
model_selection = ModelSelection(pca_features_train, y_train, pca_features_val, y_val)

ann, pred_train, pred_val = model_selection.ANN(input_dim=pca_features_train.shape[1], 
                                                output_dim=5)
performance_analysis = PerformanceAnalysis('ANN Train', pred_train, y_train)
performance_analysis.calculate_performance_metrics()
performance_analysis = PerformanceAnalysis('ANN Val', pred_val, y_val, True)
performance_analysis.calculate_performance_metrics()